In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time
from torchvision.datasets import CIFAR10
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
Tesla T4


In [ ]:
## CIFAR-10 normalization vals (3 channels, RGB)
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset  = CIFAR10(root='./data', train=False, download=True, transform=transform)

batch_size = 256

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=False,
    ##persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
    ##persistent_workers=True
)

In [ ]:
## CNN Model Definition
class CIFAR_CNN(nn.Module):
    def __init__(self): ## sets up the layers when the model is created
        super().__init__() ## calls nn.Module to properly initialize Pytorch internals

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), ## 3 input channels, 32 filters
            nn.ReLU(),

            nn.Conv2d(32, 64, kernel_size=3, padding=1), ## 32 input channels, 64 filters
            nn.ReLU(),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 32 * 32 , 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CIFAR_CNN().to(device)
print(model)

CIFAR_CNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=131072, out_features=256, bias=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 300

train_losses = []
train_accuracies = []
epoch_times = []

for epoch in range(num_epochs):
    start = time.time()
    model.train()

    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Tracking
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / len(train_loader)
    accuracy = correct / total
    elapsed = time.time() - start

    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)
    epoch_times.append(elapsed)

    print(f"Epoch {epoch+1}/{num_epochs}  "
          f"Loss: {avg_loss:.4f}  "
          f"Accuracy: {accuracy:.4f}  "
          f"Time: {elapsed:.2f}s")



Epoch 1/300  Loss: 1.4667  Accuracy: 0.4785  Time: 22.13s
Epoch 2/300  Loss: 0.9613  Accuracy: 0.6620  Time: 21.70s
Epoch 3/300  Loss: 0.7038  Accuracy: 0.7547  Time: 22.45s
Epoch 4/300  Loss: 0.4549  Accuracy: 0.8429  Time: 22.13s
Epoch 5/300  Loss: 0.2028  Accuracy: 0.9332  Time: 22.05s
Epoch 6/300  Loss: 0.0681  Accuracy: 0.9803  Time: 21.75s
Epoch 7/300  Loss: 0.0261  Accuracy: 0.9932  Time: 21.99s
Epoch 8/300  Loss: 0.0174  Accuracy: 0.9955  Time: 21.82s
Epoch 9/300  Loss: 0.0250  Accuracy: 0.9923  Time: 21.79s
Epoch 10/300  Loss: 0.0307  Accuracy: 0.9901  Time: 21.70s
Epoch 11/300  Loss: 0.0257  Accuracy: 0.9916  Time: 21.32s
Epoch 12/300  Loss: 0.0283  Accuracy: 0.9907  Time: 21.77s
Epoch 13/300  Loss: 0.0268  Accuracy: 0.9911  Time: 21.70s
Epoch 14/300  Loss: 0.0174  Accuracy: 0.9940  Time: 21.67s
Epoch 15/300  Loss: 0.0131  Accuracy: 0.9961  Time: 21.82s
Epoch 16/300  Loss: 0.0189  Accuracy: 0.9937  Time: 21.36s
Epoch 17/300  Loss: 0.0290  Accuracy: 0.9906  Time: 21.89s
Epoch 

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time
from torchvision.datasets import CIFAR10
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

In [ ]:
## CIFAR-10 normalization vals (3 channels, RGB)
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset  = CIFAR10(root='./data', train=False, download=True, transform=transform)

batch_size = 256

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=False,
    ##persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
    ##persistent_workers=True
)

In [ ]:
## CNN Model Definition
class CIFAR_CNN(nn.Module):
    def __init__(self): ## sets up the layers when the model is created
        super().__init__() ## calls nn.Module to properly initialize Pytorch internals

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), ## 3 input channels, 32 filters
            nn.ReLU(),
            nn.MaxPool2d(2), ## from 32 x 32 to 16 x 16

            nn.Conv2d(32, 64, kernel_size=3, padding=1), ## 32 input channels, 64 filters
            nn.ReLU(),
            nn.MaxPool2d(2), ##

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 2 * 2, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CIFAR_CNN().to(device)
print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 300

train_losses = []
train_accuracies = []
epoch_times = []

for epoch in range(num_epochs):
    start = time.time()
    model.train()

    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Tracking
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / len(train_loader)
    accuracy = correct / total
    elapsed = time.time() - start

    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)
    epoch_times.append(elapsed)

    print(f"Epoch {epoch+1}/{num_epochs}  "
          f"Loss: {avg_loss:.4f}  "
          f"Accuracy: {accuracy:.4f}  "
          f"Time: {elapsed:.2f}s")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time
from torchvision.datasets import CIFAR10
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

In [ ]:
## CIFAR-10 normalization vals (3 channels, RGB)
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset  = CIFAR10(root='./data', train=False, download=True, transform=transform)

batch_size = 256

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=False,
    ##persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
    ##persistent_workers=True
)

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self,channels):
        super().__init__()

        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(channels)
        ## Conv -> ReLU -> Conv -> Skip connection -> ReLU

    def forward(self, x):
         residual = x
         out = self.conv1(x)
         out = self.bn1(out)
         out = self.relu(out)
         out = self.conv2(out)
         out = out + residual ## skip connection
         out = self.relu(out) ## final ReLU
         return out

class ResNet10(nn.Module):
   def __init__(self): ## sets up the layers when the model is created
        super().__init__() ## calls nn.Module to properly initialize Pytorch internals

        self.stem = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        blocks = []
        for _ in range(10):
            blocks.append(ResidualBlock(64))
        self.res_blocks = nn.Sequential(*blocks)
        self.pool = nn.AdaptiveAvgPool2d((1,1)) ##from 64 x 32 x 32 to 64 x 1 x 1

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(64, 10)
        )

   def forward(self, x):
        x = self.stem(x) ## 64 x 32 x 32
        x = self.res_blocks(x) ## apply intial conv/ReLU
        x = self.pool(x) ## 64 x 1 x 1
        x = torch.flatten(x,1) ## just 64

        return self.classifier(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet10().to(device)
print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 300

train_losses = []
train_accuracies = []
epoch_times = []

for epoch in range(num_epochs):
    start = time.time()
    model.train()

    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Tracking
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / len(train_loader)
    accuracy = correct / total
    elapsed = time.time() - start

    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)
    epoch_times.append(elapsed)

    print(f"Epoch {epoch+1}/{num_epochs}  "
          f"Loss: {avg_loss:.4f}  "
          f"Accuracy: {accuracy:.4f}  "
          f"Time: {elapsed:.2f}s")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time
from torchvision.datasets import CIFAR10
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self,channels):
        super().__init__()

        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        ## Conv -> ReLU -> Conv -> Skip connection -> ReLU

    def forward(self, x):
         residual = x
         out = self.conv1(x)
         out = self.relu(out)
         out = self.conv2(out)
         out = out + residual ## skip connection
         out = self.relu(out) ## final ReLU
         return out

class ResNet10(nn.Module):
   def __init__(self): ## sets up the layers when the model is created
        super().__init__() ## calls nn.Module to properly initialize Pytorch internals

        self.stem = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

        blocks = []
        for _ in range(10):
            blocks.append(ResidualBlock(64))
        self.res_blocks = nn.Sequential(*blocks)
        self.pool = nn.AdaptiveAvgPool2d((1,1)) ##from 64 x 32 x 32 to 64 x 1 x 1

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.3), ## dropout
            nn.Linear(64,10)
        )


   def forward(self, x):
        x = self.stem(x) ## 64 x 32 x 32
        x = self.res_blocks(x) ## apply intial conv/ReLU
        x = self.pool(x) ## 64 x 1 x 1
        x = torch.flatten(x,1) ## just 64

        return self.classifier(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet10().to(device)
print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 300

train_losses = []
train_accuracies = []
epoch_times = []

for epoch in range(num_epochs):
    start = time.time()
    model.train()

    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Tracking
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / len(train_loader)
    accuracy = correct / total
    elapsed = time.time() - start

    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)
    epoch_times.append(elapsed)

    print(f"Epoch {epoch+1}/{num_epochs}  "
          f"Loss: {avg_loss:.4f}  "
          f"Accuracy: {accuracy:.4f}  "
          f"Time: {elapsed:.2f}s")